<a></a>
# ResNet - Skin Lesions Image Classification with ISIC 2019

Access Drive which contains the Dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Variables Initialization


In [ ]:
# Train fcc
second = False
epochs = 60 #sse mudo aqui tenho que mudar fcc o lr no_epochs=50
# freeze = false - means initialization but not freeze
freeze = False


### Filepath saved weights 
random_rot = False # true = no weights loaded from the previous task
random_simclr = False

earlyfusion = False # True = early-- false = latefsion

## Late fusion weights
# Path to classification model saved
filepath_rot_class =  '/content/drive/MyDrive/Tese/fold1_eval_rot_imagenet_bempr_crot'
filepath_simclr_class = '/content/drive/MyDrive/Tese/fold2_simclr_hflip_crop_rot_imagw_prbem_eval_valid.h5'


# Path to save new classification model
filepath_fcc = '/content/drive/MyDrive/Tese/fold4_eval_latefusion_imagenet'
filepath_fcc_loss = '/content/drive/MyDrive/Tese/fold4_latefusion_eval_rot_imagenet'

#----------------------------------
### Network 
weight_resnet = True #true =imagenet
res = 'old'
# res = 'new'
## Resnet50
rede = 'resnet50' 
h1 = 256
h2 = 128


#----------------------------------
### Evaluation method
aval = 'fcc'
extra_mlp = False # true if we want fcc after the gap and before softmax
h1_fcc = 512
h2_fcc = 128

# aval = 'svm'
param_c = 100
param_g = 0.1

#----------------------------------
### Feature Space
fspace = 'gap'  #2048 / 64
# fspace = 'h2' #128 / 32
# fspace = 'h1' #256 / 64



In [ ]:
if second == False:
    !unzip /content/drive/MyDrive/Tese/ISIC_2019_pre-processed_fold4.zip -d /content

<a></a>
## 1.Imports

In [ ]:
import csv
import pandas as pd
import numpy as np
import os,sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from collections import Counter

sys.path.append('/content/drive/MyDrive/Tese/')

from visuals import plot_grouped_2bars
import math
import shutil
from sklearn.utils import shuffle

import glob

import os, os.path
from ast import literal_eval
from make_dir_versao_writedisco import *

import cv2

from PIL import Image


import keras
from tensorflow.keras.layers import Input, Dense, Activation,Flatten, Convolution2D, Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras import layers, losses

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
#from tensorflow.python.keras.preprocessing.image import image_dataset_from_directory

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

from mpl_toolkits.mplot3d import Axes3D

from tensorflow.keras.models import Sequential, save_model, load_model

from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import GridSearchCV , StratifiedShuffleSplit
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report

import seaborn as sns 
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,TensorBoard
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

NUM_TRAIN_SAMPLES = 17731
NUM_VAL_SAMPLES = 7600
BATCH_SIZE = 32



<a></a>
## 2. Online Data Augmentation

In [ ]:
from tensorflow.python.keras.applications.resnet import ResNet50, preprocess_input
datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=90,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        #rescale=1./255,
        #shear_range=0.2,
        #brightness_range=[0.2,1],
        vertical_flip = True,
        horizontal_flip=True)

datagenV = ImageDataGenerator(
    preprocessing_function=preprocess_input,
        rotation_range=90,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        #rescale=1./255,
        #shear_range=0.2,
        #brightness_range=[0.2,1],
        vertical_flip = True,
        horizontal_flip=True)


seed = 1
train_generator = datagen.flow_from_directory(
    directory = '/content/ISIC_2019_pre-processed_fold4/train',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = True,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)
seed = 2
validation_generator = datagenV.flow_from_directory(
    directory = '/content/ISIC_2019_pre-processed_fold4/valid',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = True,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)

seed = 3
trainPredict_generator = datagen.flow_from_directory(
    directory = '/content/ISIC_2019_pre-processed_fold4/train',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = False,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)
seed = 4
validationPredict_generator = datagenV.flow_from_directory(
    directory =  '/content/ISIC_2019_pre-processed_fold4/valid',
    target_size = (224,224),
    color_mode = 'rgb',
    shuffle = False,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    seed = seed)


Found 17731 images belonging to 8 classes.
Found 7600 images belonging to 8 classes.
Found 17731 images belonging to 8 classes.
Found 7600 images belonging to 8 classes.


<a></a>
## 3.Build ResNet Architecture + Predict classification results


In [ ]:
print('LATE FUSION')
from tensorflow.keras.applications.resnet import ResNet50
if weight_resnet == True:
  MODEL = ResNet50(include_top=False, weights= 'imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None)
  # MODEL.summary()
else:
  MODEL = ResNet50(include_top=False, weights= None, input_tensor=None, input_shape=(224,224,3), pooling=None)
  # MODEL.summary()


###########################################################################
## Rotation model
MODEL = ResNet50(include_top=False, weights= 'imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None)
# MODEL.summary()
resnet = Model(inputs=MODEL.input, outputs=MODEL.output,name='resnet50_rot')
inp = Input((224,224,3))

# x=tf.keras.applications.resnet.preprocess_input(inputs)
model_rot = resnet(inp)
# Flatten the output layer to 1 dimension
gap = GlobalAveragePooling2D()(model_rot)
model_rot = Dropout(0.5)(gap)
dense_rot = Dense(8,activation = 'softmax')(model_rot)


model_rot = Model(inputs=inp, outputs=dense_rot)
model_rot.summary()

if random_rot == False:
  model_rot.load_weights(filepath_rot_class)
  print('\n\nloaded weights from rot')


#########################################################################
## SIMCLR  
MODEL_simclr = ResNet50(include_top=False, weights= None, input_tensor=None, input_shape=(224,224,3), pooling=None)

h = MODEL_simclr(inp)
gap_simclr = GlobalAveragePooling2D()(h)
drop_simclr = Dropout(0.5)(gap_simclr)
dense_simclr = Dense(8,activation = 'softmax')(drop_simclr)

model_simclr = Model(inp, dense_simclr)
model_simclr.summary()

if random_simclr == False:
    model_simclr.load_weights(filepath_simclr_class)
    print('\n\nloaded weights from simclr')

## Late Fusion
x = concatenate([dense_rot,dense_simclr])
merge1 = Dropout(0.5)(x)
softmax = Dense(8,activation = 'softmax')(merge1)


# classical
model = Model(inputs=inp, outputs=softmax)   
model.summary()


# Since encoder is trained freeze the weights of encoder and just train fully connected part
for layer in model.layers[0:(len(model.layers)-1)]:
    layer.trainable = False

model.summary()

optimizer = tf.keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, decay=0.0)
model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])



In [ ]:
print('train simclr')
Y_pred_simclr = model_simclr.predict_generator(trainPredict_generator)
print('train rot')
Y_pred_rot = model_rot.predict_generator(trainPredict_generator)

final_t = np.mean([Y_pred_simclr,Y_pred_rot], axis=0)
# print(final)
y_pred_t = np.argmax(final_t, axis=1)
# print(y_pred)


<a></a>
## 4. Analise the Trained model

In [ ]:

print('Confusion Matrix Train')
validConfMatrix =confusion_matrix(trainPredict_generator.classes, y_pred_t)
print('Accuracy train: '+str(accuracy_score(trainPredict_generator.classes, y_pred_t)))
print(validConfMatrix)
print(classification_report(trainPredict_generator.classes, y_pred_t))#Output

diag = 0
vec_val = [1357,3863,997,260,787,72,76,188]
vec_train = [3165,9012,2326,607,1837,167,177,440]
for col in range(0, 8):
  for lin in range(0, 8):
      if col == lin:
        diag = (diag + validConfMatrix[lin][col]/vec_train[col])
diag=diag/8
print('Diagonal mean value:')
print(diag)

print('\nvalid')
Y_pred_simclr_v = model_simclr.predict_generator(validationPredict_generator)
print(Y_pred_simclr_v)
Y_pred_rot_v = model_rot.predict_generator(validationPredict_generator)
print(Y_pred_rot_v)
final_v = np.mean([Y_pred_simclr_v,Y_pred_rot_v], axis=0)
print(final_v)
y_pred_v = np.argmax(final_v, axis=1)
print(y_pred_v)

print('Confusion Matrix Validation')
validConfMatrix =confusion_matrix(validationPredict_generator.classes, y_pred_v)
print('Accuracy valid: '+str(accuracy_score(validationPredict_generator.classes, y_pred_v)))
print(validConfMatrix)
print(classification_report(validationPredict_generator.classes, y_pred_v,digits=4))#Output

diag = 0
vec_val = [1357,3863,997,260,787,72,76,188]
vec_train = [3165,9012,2326,607,1837,167,177,440]
for col in range(0, 8):
  for lin in range(0, 8):
      if col == lin:
        diag = (diag + validConfMatrix[lin][col]/vec_val[col])
diag=diag/8
print('Diagonal mean value:')
print(diag)

In [ ]:
FP = [0,0,0,0,0,0,0,0]
TN = [0,0,0,0,0,0,0,0]
SP = [0,0,0,0,0,0,0,0]
        
for lesion in range(0,8):
    for i in range(0, 8):
        if i != lesion:
            FP[lesion] = FP[lesion] + int(validConfMatrix[i][lesion])

    for i in range(0, 8):
        for j in range(0, 8):
            if i != lesion and j != lesion:
                TN[lesion] = TN[lesion] + int(validConfMatrix[i][j])

for i in range(0,8):
    SP[i] = TN[i]/(TN[i]+FP[i])

In [ ]:
SP

In [ ]:
sum(SP)/len(SP)